In [1]:
import os
import json
#from pandas import json_normalize
import pandas as pd
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
file = pd.read_csv("nyt_concatenated_data.csv")

In [ ]:
headline_emotion = []
lead_paragraph_emotion = []
headline_sentiment = []
lead_paragraph_sentiment = []

In [ ]:
# Get the Tone Analyzer API Key and URL
tone_api = os.getenv("tone_key")
tone_url = os.getenv("tone_url")

In [ ]:
authenticator = IAMAuthenticator(tone_api)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

natural_language_understanding.set_service_url(tone_url)
for i, j in file.iterrows():
    response = natural_language_understanding.analyze(
        text= j.Headline,
        features=Features(emotion =EmotionOptions(), sentiment=SentimentOptions())).get_result()
#print(json.dumps(response, indent=2))
    headline_emotion.append(response['emotion']['document']['emotion'])
    headline_sentiment.append(response['sentiment']['document'])

In [ ]:
natural_language_understanding.set_service_url(tone_url)
for i, j in file.iterrows():
    response = natural_language_understanding.analyze(
        text= j['Lead Paragraph'],
        features=Features(emotion =EmotionOptions(), sentiment=SentimentOptions())).get_result()
#print(json.dumps(response, indent=2))
    lead_paragraph_emotion.append(response['emotion']['document']['emotion'])
    lead_paragraph_sentiment.append(response['sentiment']['document'])

In [9]:
df_lead_paragraph_sent = pd.DataFrame(lead_paragraph_sentiment)
df_lead_paragraph_emotion = pd.DataFrame(lead_paragraph_emotion)

In [20]:
df_lead_paragraph_sent.rename(columns={"label": "emotion_lead_par", "score": "score_lead_par"}, inplace = True)
df_lead_paragraph_emotion.rename(columns = {"anger":"anger_lead_par","disgust":"disgust_lead_par", "fear" :"fear_lead_par", "joy": "joy_lead_par", "sadness": "sadness_lead_par"}, inplace = True)

In [24]:
df_headline_sent = pd.DataFrame(headline_sentiment)
df_headline_emotion = pd.DataFrame(headline_emotion)

In [25]:
df_headline_sent.rename(columns={"label": "emotion_headline", "score": "score_headline"}, inplace = True)
df_headline_emotion.rename(columns = {"anger":"anger_headline","disgust":"disgust_headline", "fear" :"fear_headline", "joy": "joy_headline", "sadness": "sadness_headline"}, inplace = True)


In [30]:
df_concat1 = pd.concat([df_headline_sent,df_headline_emotion], join = 'inner', axis = 1)
df_concat2 = pd.concat([df_lead_paragraph_sent, df_lead_paragraph_emotion], join = 'inner', axis =1)
df_concat3 = pd.concat([df_concat1, df_concat2], join = 'inner', axis =1)
df_nlp = pd.concat([df_concat3, file], join = 'inner', axis = 1)
df_nlp.set_index('Date', inplace = True)

In [32]:
df_nlp.to_csv(r'df_nlp.csv', index = True)